# Lab. 2-1 Schema Preparation-2

일반적인 Schema Linking 과정이 테이블 선택 -> 컬럼 선택으로 나눠 진행되는데, 테이블 선택이 잘못되면 후속 과정은 무의미하기 때문에 각 테이블에 대한 충분한 설명을 갖추는 것이 중요합니다.

이 노트북에서는 아래의 table_name, table_desc, columns, col_name, col_desc, table_summary, table_summary_v 를
OpenSearch 의 schema_descriptions 인덱스에 적재하는 일을 합니다.

---
```

table_doc = {
    "table_name": Customer,
    "table_desc": 고객 세부 정보를 포함하고 지원 담당자에게 연결합니다,
    "columns": [
    {
      "col_name": "CustomerId",
      "col_desc": "기본 키, 고유한 고객 식별자입니다."
    },
    {
      "col_name": "FirstName",
      "col_desc": "고객의 이름입니다."
    },
    ...
    "table_summary": 이 테이블은 고객의 세부 정보를 포함하고 있으며, 지원 담당자와 연결됩니다. 다음과 같은 정보를 포함하고 있습니다:
        - 고유한 고객 식별자(CustomerId)
        - 고객의 이름(FirstName, LastName) 
        - 고객의 회사 정보(Company)
        - 고객의 주소 정보(Address, City, State, Country, PostalCode)
        - 고객의 연락처 정보(Phone, Fax, Email)
        - 고객을 지원하는 직원 정보(SupportRepId)
        이 테이블을 활용하여 다음과 같은 분석 및 활용이 가능합니다:
        - 특정 국가 또는 지역 고객 데이터 추출 및 분석 (예: 캐나다, 미국, 브라질 고객 데이터 분석)
    "table_summary_v": [-0.062046438455581665,0.01113771554082632,...]
```



---

아래의 전체 구성도에서 다음과 같은 단계별 작업을 합니다.
- 3.Table Summarizer --> Schema Descriptions with Table Summaries -->  Schema Descriptions with Table Summaries Embeeding --> 1) Schema Document --> Amazon OpenSearch Service 저장

![Intro](../images/text2sql/schema-prep-1.png)

## 1. OpenSearch 환경 설정

In [1]:
import sys
from libs.ssm import parameter_store

pm = parameter_store('us-east-1')
# pm = parameter_store('us-west-2')
domain_endpoint = pm.get_params(key="chatbot-opensearch_domain_endpoint", enc=False)
opensearch_domain_endpoint = f"https://{domain_endpoint}"
opensearch_user_id = pm.get_params(key="chatbot-opensearch_user_id", enc=False)
opensearch_user_password = pm.get_params(key="chatbot-opensearch_user_password", enc=True)
print("opensearch_domain_endpoint: \n", opensearch_domain_endpoint)

opensearch_domain_endpoint: 
 https://search-text2sql-kh-ujitj6xavl2cfvpkafvolvjjlm.us-east-1.es.amazonaws.com


## 2. Schema Description 및 Example Queries 로드

In [2]:
import json 
SCHEMA_FILE_PATH = "database/chinook_schema_kr.json"
SAMPLE_QUERY_FILE_PATH = "database/example_queries_temp.jsonl"

def load_schema(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        schema = json.load(file)
    return schema

def load_queries(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        queries = file.readlines()
    return queries

schema = load_schema(SCHEMA_FILE_PATH)
queries = load_queries(SAMPLE_QUERY_FILE_PATH)

### 2.1. Schema 로드

In [3]:
def print_pretty_json(data):
    print(json.dumps(data, indent=4, ensure_ascii=False))

print_pretty_json(schema)

[
    {
        "Album": {
            "table_desc": "고유 ID, 제목 및 아티스트 ID를 통한 아티스트 링크가 있는 앨범 데이터를 저장합니다.",
            "cols": [
                {
                    "col": "AlbumId",
                    "col_desc": "기본 키, 앨범의 고유 식별자입니다."
                },
                {
                    "col": "Title",
                    "col_desc": "앨범의 제목입니다."
                },
                {
                    "col": "ArtistId",
                    "col_desc": "앨범의 아티스트를 참조하는 외래 키입니다."
                }
            ]
        },
        "Artist": {
            "table_desc": "ID와 이름이 있는 아티스트 정보를 보유합니다.",
            "cols": [
                {
                    "col": "ArtistId",
                    "col_desc": "기본 키, 아티스트의 고유 식별자입니다."
                },
                {
                    "col": "Name",
                    "col_desc": "아티스트의 이름입니다."
                }
            ]
        },
        "Customer": {
            "table_desc": "고객 세부 정보를 포함하고 지원 담당자에게 연결합니다.",
         

### 2.2. 자연어 질문 & SQL 로드

In [4]:
import json
def print_pretty_json_in_list(data, n_sample):
    # 각 JSON 문자열을 파싱하고 이쁘게 출력
    for i, item in enumerate(data):
        # 문자열을 JSON 객체로 파싱
        parsed = json.loads(item)
        if i < n_sample:
            # indent=2로 설정하여 이쁘게 출력
            pretty_json = json.dumps(parsed, indent=2, ensure_ascii=False)
            print(pretty_json)
            print("-" * 50)  # 구분선 출력

print_pretty_json_in_list(queries, 3)        

{
  "input": "모든 아티스트의 ID와 이름 정보 조회. 신규 아티스트 데이터베이스 구축, 아티스트 전체 명단 확인, 아티스트 관련 분석 및 보고서 작성 등의 목적으로 활용 가능합니다.",
  "query": "SELECT * FROM Artist"
}
--------------------------------------------------
{
  "input": "아티스트 이름이 'AC/DC'인 아티스트의 모든 앨범 정보 요청\n\n이 쿼리는 특정 아티스트의 모든 앨범 정보를 검색하는 데 사용됩니다. 아티스트 테이블에서 'AC/DC'라는 이름을 가진 아티스트의 ArtistId를 찾은 다음, 해당 ArtistId를 Album 테이블에서 검색하여 관련 앨범 정보를 가져옵니다. 이를 통해 음악 스트리밍 서비스나 온라인 음악 상점에서 특정 아티스트의 전체 앨범 목록을 표시하거나 아티스트 프로필 페이지에 앨범 정보를 포함시킬 수 있습니다.",
  "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC')"
}
--------------------------------------------------
{
  "input": "록 장르에 속하는 모든 트랙 정보 조회. 장르 이름을 기준으로 해당 장르 ID를 찾아 이를 활용해 Track 테이블에서 일치하는 GenreId를 가진 모든 레코드를 반환합니다. 사용자가 특정 장르의 음악을 탐색하거나 재생 목록을 만들고자 할 때 유용한 쿼리입니다. 장르별 트랙 정보를 효율적으로 조회할 수 있어 음악 스트리밍 서비스나 미디어 플레이어 애플리케이션에서 활용 가능합니다.",
  "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock')"
}
------------------------------------

## 3. 테이블 요약 문서 생성

### 3.1. 요약 프롬프트 생성
- 다양한 정보들을 테이블 요약 문서 생성에 활용합니다. 
- 기본 Schema Description 문서와 Sample Query 등을 모두 활용해서 테이블 요약을 생성합니다. 
- 아래는 이 정보를 반영하기 위한 LLM 프롬프트 템플릿입니다.

In [5]:
from langchain_aws import ChatBedrock

SYS_PROMPT = """
You are a data analyst that can help summarize SQL tables.
Summarize the provided table by the given context.

<instruction>
- You shall write the summary based only on the provided information, and make it as detailed as possible.
- Note that above sampled queries are only small sample of queries and thus not all possible use of tables are represented, and only some columns in the table are used.
- Do not use any adjective to describe the table. For example, the importance of the table, its comprehensiveness or if it is crucial, or who may be using it. For example, you can say that a table contains certain types of data, but you cannot say that the table contains a 'wealth' of data, or that it is 'comprehensive'.
- Do not mention about the sampled query. Only talk objectively about the type of data the table contains and its possible utilities.
- Please also include some potential usecases of the table, e.g. what kind of questions can be answered by the table, what kind of anlaysis can be done by the table, etc.
- Please provide the output in Korean.
</instruction>
"""

PROMPT_TEMPLATE = """
<table schema>
{table_schema}
</table schema>

<sample queries>
{sample_queries}
</sample queries>
"""

### 3.2. LangChain Bedrock 그리고 서치 함수등 정의

In [6]:
model_kwargs =  { 
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    "system": SYS_PROMPT
}

chat_model = ChatBedrock(
    model_id="us.anthropic.claude-3-sonnet-20240229-v1:0",
    region_name='us-east-1',
    # region_name='us-west-2',
    model_kwargs=model_kwargs
)

In [7]:
def search_table_queries(queries, table_name):  # 테이블이 어떤 쿼리에 사용되었는지 검색하여 추출하는 함수입니다.
    table_name_lower = table_name.lower()
    matched_queries = []

    for line in queries:
        try:
            query_data = json.loads(line)
            if table_name_lower in query_data['query'].lower():
                matched_queries.append(query_data)
        except json.JSONDecodeError:
            print(f"Invalid JSON line: {line}")
    
    return matched_queries
    
import json
from pprint import pprint

def pretty_print_queries(data, n_sample):
    for idx, query in enumerate(data, 1):
        if idx < n_sample:
            print(f"\n[Query {idx}]")
            print(json.dumps(query, indent=2, ensure_ascii=False))
            print("-" * 80)    

### 3.3. 전체 "자연어-SQL" 로 부터 `Customer`라는 단어가 있는 "자연어-SQL" 추출 

In [8]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

table_name = 'Customer'

matched_queries = search_table_queries(queries, table_name)
prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
print("## table_name: ", table_name)
print("## matched_queries: ")
pretty_print_queries(matched_queries, 3)

## table_name:  Customer
## matched_queries: 

[Query 1]
{
  "input": "캐나다에 거주하는 고객들의 전체 세부 정보 조회. 캐나다 시장 분석 및 맞춤형 마케팅 전략 수립을 위해 해당 국가 고객 데이터가 필요할 수 있습니다.",
  "query": "SELECT * FROM Customer WHERE Country = 'Canada'"
}
--------------------------------------------------------------------------------

[Query 2]
{
  "input": "고객별 총 구매 금액을 내림차순으로 정렬하여 상위 5명의 고객 ID와 총 구매 금액 조회. 고객 관계 관리 및 마케팅 전략 수립을 위해 최대 지출 고객 파악이 필요할 수 있음. 또한 VIP 고객 관리 프로그램 운영 시 대상 고객 선정에 활용 가능함.",
  "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5"
}
--------------------------------------------------------------------------------


### 3.4. 변수인 matched_queries 와 테이블 이름을 입력한 프롬프트 확인

In [9]:
print(SYS_PROMPT)
print(prompt.format(sample_queries=matched_queries, table_schema=schema[0]['Customer']))


You are a data analyst that can help summarize SQL tables.
Summarize the provided table by the given context.

<instruction>
- You shall write the summary based only on the provided information, and make it as detailed as possible.
- Note that above sampled queries are only small sample of queries and thus not all possible use of tables are represented, and only some columns in the table are used.
- Do not use any adjective to describe the table. For example, the importance of the table, its comprehensiveness or if it is crucial, or who may be using it. For example, you can say that a table contains certain types of data, but you cannot say that the table contains a 'wealth' of data, or that it is 'comprehensive'.
- Do not mention about the sampled query. Only talk objectively about the type of data the table contains and its possible utilities.
- Please also include some potential usecases of the table, e.g. what kind of questions can be answered by the table, what kind of anlaysis c

### 3.5. LLM 호출하여 테이블 요약 정보 얻기

In [10]:
chain = prompt | chat_model | StrOutputParser()

table_summary = chain.invoke({"table_schema": schema[0]['Customer'], "sample_queries": matched_queries})
print(table_summary)

이 테이블은 고객의 세부 정보와 지원 담당자 정보를 포함하고 있습니다. 고객 ID, 이름, 회사명, 주소, 도시, 주/도, 국가, 우편번호, 전화번호, 팩스번호, 이메일 주소 등의 고객 정보와 함께 해당 고객을 지원하는 직원의 ID가 포함되어 있습니다.

이 테이블을 활용하면 다음과 같은 분석 및 활용이 가능합니다:

- 특정 국가 또는 지역에 거주하는 고객 데이터를 필터링하여 해당 시장 분석 및 맞춤형 마케팅 전략 수립에 활용 가능합니다.
- 고객별 총 구매 금액을 계산하여 최대 지출 고객을 파악하고, VIP 고객 관리 프로그램 운영 시 대상 고객 선정에 활용할 수 있습니다. 
- 회사 정보가 있는 고객 데이터를 추출하여 기업 고객 관리, 영업 및 마케팅 활동에 활용할 수 있습니다.
- 지원 담당 직원별 담당 고객 수와 매출 실적을 분석하여 직원 성과 평가 및 인센티브 제공에 활용할 수 있습니다.
- 고객의 연락처 정보를 활용하여 고객 커뮤니케이션 및 마케팅 활동에 활용할 수 있습니다.

이 테이블은 고객 관계 관리, 마케팅 전략 수립, 영업 활동, 직원 성과 관리 등 다양한 비즈니스 활동에 필요한 고객 데이터를 제공합니다.


### 3.6. 테이블 요약 생성을 모든 테이블에 대해 수행
- Schema Description 내에 모든 테이블 수행
- 약 1~2분 소요됩니다

In [11]:
%%time 

import os

OUTPUT_FILE_PATH1 = "database/chinook_detailed_schema_temp.json"

with open(OUTPUT_FILE_PATH1, 'w', encoding='utf-8') as output_file:
    output_file.write('[\n')

def summarize_table(table_name, table_data, queries, chain):
    table_summary = chain.invoke({"table_schema": table_data, "sample_queries": queries})
    table_data['table_summary'] = table_summary 
    summary_output = {table_name: table_data}
    return summary_output
    
for table_info in schema:
    for table_name, table_data in table_info.items():
        globals()[table_name] = table_data
        matched_queries = search_table_queries(queries, table_name)
        prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
        chain = prompt | chat_model | StrOutputParser()

        table_summary = summarize_table(table_name, table_data, matched_queries, chain)
        
        with open(OUTPUT_FILE_PATH1, 'a', encoding='utf-8') as output_file:
            output_file.write(json.dumps(table_summary, ensure_ascii=False, indent=4) + ',\n')

with open(OUTPUT_FILE_PATH1, 'rb+') as output_file:
    output_file.seek(-2, os.SEEK_END) 
    output_file.truncate() 
    output_file.write(b'\n]')

CPU times: user 53.3 ms, sys: 8.83 ms, total: 62.1 ms
Wall time: 1min 43s


이제 `database/chinook_detailed_schema_temp.json` 파일을 열어보면, table_summary가 스키마 문서에 추가되어 있습니다.

위와 같이, 1) 테이블에 어떤 컬럼들이 있는지, 2) 어떤 용도로 활용되는지에 대한 자세한 정보를 LLM에 전달하는 것은 올바른 테이블 선택에 도움이 됩니다.

하지만, 테이블 요약이 너무 길어졌을 때 모든 테이블의 요약 정보를 LLM에 전달할 수 없으므로, 테이블 요약 정보 역시 벡터 유사도 검색으로 탐색하는 것이 좋습니다.

## 4. OpenSearch: 테이블 요약 문서를 벡터 임베딩으로 변환하여 OpenSearch에 저장

### 4.1. schema_descriptions 이름의 인덱스 생성
- schema_descriptions 인덱스가 존재하면 지우고, 다시 생성 함

In [12]:
import yaml
from opensearchpy import OpenSearch, RequestsHttpConnection
INDEX_NAME = "schema_descriptions"

def load_opensearch_config():
    with open("./libs/opensearch_fix.yml", 'r', encoding='utf-8') as file:
        return yaml.safe_load(file)

def init_opensearch(config):
    mapping = {"settings": config['settings'], "mappings": config['mappings-detailed-schema']}
    endpoint = opensearch_domain_endpoint
    http_auth = (opensearch_user_id, opensearch_user_password)

    os_client = OpenSearch(
            hosts=[{'host': endpoint.replace("https://", ""),'port': 443}],
            http_auth=http_auth, 
            use_ssl=True,
            verify_certs=True,
            timeout=300,
            connection_class=RequestsHttpConnection
    )

    create_os_index(os_client, mapping)
    return os_client

def create_os_index(os_client, mapping):
    exists = os_client.indices.exists(INDEX_NAME)

    if exists:
        os_client.indices.delete(index=INDEX_NAME)
        print("Existing index has been deleted. Create new one.")
    else:
        print("Index does not exist, Create one.")

    os_client.indices.create(INDEX_NAME, body=mapping)

config = load_opensearch_config()
os_client = init_opensearch(config)

Existing index has been deleted. Create new one.


### 4.2. table_summary 의 table_summary_v 임베딩 값 구하기

In [13]:
from langchain_aws import BedrockEmbeddings

emb_model = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0", region_name='us-east-1', model_kwargs={"dimensions":1024}) 
OUTPUT_FILE_PATH2 = "database/chinook_detailed_schema.json"

def embedding_summary(emb_model):
    with open(OUTPUT_FILE_PATH1, 'r', encoding='utf-8') as input_file:
        data_list = json.load(input_file)

    for data in data_list:
        table_name = list(data.keys())[0]
        table_summary = data[table_name]["table_summary"]
        data[table_name]["table_summary_v"] = emb_model.embed_query(table_summary)
    
    with open(OUTPUT_FILE_PATH2, 'w', encoding='utf-8') as output_file:
        json.dump(data_list, output_file, ensure_ascii=False, indent=4)

embedding_summary(emb_model)

### 4.4. 변환된 table_summary_v 벡터 임베딩 확인 
- 이제 `database/chinook_detailed_schema_temp.json` 파일을 열어보면, table_summary 및 이에 대한 임베딩이 스키마 문서에 추가되어 있습니다.

### 4.5. 오픈 서치에 Bulk Insert

In [14]:
def load_detailed_schema_descriptions(os_client):

    with open(OUTPUT_FILE_PATH2, 'r') as file:
        schema_data = json.load(file)

    bulk_data = []
    for table in schema_data:
        for table_name, table_info in table.items():
            table_doc = {
                "table_name": table_name,
                "table_desc": table_info["table_desc"],
                "columns": [{"col_name": col["col"], "col_desc": col["col_desc"]} for col in table_info["cols"]],
                "table_summary": table_info["table_summary"],
                "table_summary_v": table_info["table_summary_v"]
            }
            bulk_data.append({"index": {"_index": INDEX_NAME, "_id": table_name}})
            bulk_data.append(table_doc)
    
    bulk_data_str = '\n'.join(json.dumps(item) for item in bulk_data) + '\n'

    response = os_client.bulk(body=bulk_data_str)
    if response["errors"]:
        print("There were errors during bulk indexing:")
        for item in response["items"]:
            if 'index' in item and item['index']['status'] >= 400:
                print(f"Error: {item['index']['error']['reason']}")
    else:
        print("Bulk-inserted all items successfully.")

    return response

response = load_detailed_schema_descriptions(os_client)

Bulk-inserted all items successfully.


### 4.6. 오픈 서치 저장 결과 확인

In [15]:
top_3_items = response['items'][:3]
for item in top_3_items:
    print(item)

{'index': {'_index': 'schema_descriptions', '_id': 'Album', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}
{'index': {'_index': 'schema_descriptions', '_id': 'Artist', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}
{'index': {'_index': 'schema_descriptions', '_id': 'Customer', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}


In [16]:
# 인덱스의 전체 레코드 수 확인
def count_all_records(os_client, index_name):
    try:
        # count API 사용하여 인덱스의 총 문서 수 확인
        count_response = os_client.count(index=index_name)
        total_records = count_response['count']
        print(f"인덱스 '{index_name}'에 총 {total_records}개의 레코드가 있습니다.")
        return total_records
    except Exception as e:
        print(f"레코드 수 확인 중 오류 발생: {str(e)}")
        return None

# 사용 예시
index_name = "schema_descriptions"  # 실제 인덱스 이름으로 변경하세요
total_count = count_all_records(os_client, index_name)

인덱스 'schema_descriptions'에 총 11개의 레코드가 있습니다.


#### Customer Table 의 테이블/컬럼 설명, 테이블 요약 정보, 테이블 요약 정보의 Vector 값 

![schema_desc.png](img/schema_desc.png)